In [1]:
import requests
import string

In [2]:
req = requests.get('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

In [3]:
req

<Response [200]>

In [4]:
dat = req.text.split('\n')

In [5]:
dat[0]

'First Citizen:'

In [6]:
data = " ".join(dat)

In [7]:
data[:100]

'First Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.  First Citizen: You'

In [8]:
print(str.maketrans("","",string.punctuation))

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [9]:
len(data)

1115394

In [10]:
def new_text(data):
    table = str.maketrans("","",string.punctuation)
    data = data .translate(table)
    return data

In [11]:
data = new_text(data)

In [12]:
from nltk.tokenize import word_tokenize

In [13]:
tokens = word_tokenize(data)

In [14]:
tokens[:15]

['First',
 'Citizen',
 'Before',
 'we',
 'proceed',
 'any',
 'further',
 'hear',
 'me',
 'speak',
 'All',
 'Speak',
 'speak',
 'First',
 'Citizen']

In [15]:
len(tokens)

202837

In [16]:
n = 40 + 1
lines = []
for i in range(n, len(tokens)):
    seq = tokens[i - n : i]
    line = ' '.join(seq)
    lines.append(line)

In [17]:
len(lines)

202796

In [18]:
lines[0]

'First Citizen Before we proceed any further hear me speak All Speak speak First Citizen You are all resolved rather to die than to famish All Resolved resolved First Citizen First you know Caius Marcius is chief enemy to the people'

In [19]:
len(lines[0].split())

41

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
tokenizer = Tokenizer()

In [22]:
tokenizer.fit_on_texts(lines)

In [23]:
sequences = tokenizer.texts_to_sequences(lines)

In [24]:
sequences = np.asarray(sequences)

In [25]:
sequences.shape

(202796, 41)

In [26]:
sequences[0]

array([  91,  270,  137,   35, 1003,  142,  675,  124,   15,  107,   33,
        107,  107,   91,  270,    6,   40,   33, 1402,  352,    3,  199,
         63,    3, 3866,   33, 1402, 1402,   91,  270,   91,    6,   93,
       1197,  224,   12, 2488,  594,    3,    1,  305])

In [27]:
X, y = sequences[:, :-1], sequences[:, -1]

In [28]:
X.shape

(202796, 40)

In [29]:
y.shape

(202796,)

In [30]:
X[0]

array([  91,  270,  137,   35, 1003,  142,  675,  124,   15,  107,   33,
        107,  107,   91,  270,    6,   40,   33, 1402,  352,    3,  199,
         63,    3, 3866,   33, 1402, 1402,   91,  270,   91,    6,   93,
       1197,  224,   12, 2488,  594,    3,    1])

In [31]:
y[0]

305

In [32]:
vocab_size = len(tokenizer.word_index) + 1

In [33]:
vocab_size

12848

In [34]:
y = to_categorical(y, num_classes = vocab_size)

MemoryError: Unable to allocate 9.71 GiB for an array with shape (202796, 12848) and data type float32

In [35]:
y.shape

(202796,)

In [36]:
y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [37]:
seq_length = X.shape[1]

In [38]:
seq_length

40

In [39]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 50)            642400    
_________________________________________________________________
lstm (LSTM)                  (None, 40, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 12848)             1297648   
Total params: 2,090,948
Trainable params: 2,090,948
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X,y,batch_size = 256, epochs = 2)

Epoch 1/2
793/793 [==============================] - 423s 534ms/step - loss: 6.8746 - accuracy: 0.0312
Epoch 2/2
498/793 [=================>............] - ETA: 2:28 - loss: 6.5410 - accuracy: 0.0402

In [1]:
text = ["First Citizen Before we proceed any further hear me speak All Speak speak First Citizen You are all resolved rather to die than to famish All Resolved resolved First Citizen First you know Caius Marcius is chief enemy to the people"]

In [ ]:
tokenizer.fit_on_texts(text)

In [ ]:
sequences = tokenizer.texts_to_sequences(text)
sequences = np.asarray(sequences)

In [ ]:
sequences.shape

In [ ]:
sequences[0]

In [ ]:
text_X = sequences

In [ ]:
model.predict(test_X)

In [ ]:
np.argmax(pred)

In [ ]:
len(pred[0])

In [ ]:
tokenizer.word_indexrd